### 1. Feature Selection 란?
- 모델을 구성하는 주요 피처들을 선택
- 불필요한 다수의 피처들로 인해 모델 성능 저하 가능성 제거
- 다수의 피처로 모델 학습시 overfitting 이슈 발생 제거

#### (방법)
- 1) Filter Method :각각의 변수들에 대해 통계적인 점수를 부여하여 부여된 점수를 바탕으  로 변수 순위를 매겨 변수를 선택하는 방법
correlation coefficient,
- 2) Wrapper Method  ( Subset Selection )
변수간 상호작용을 감지할 수 있도록 변수의 일부만을 모델링에 사용한  후 그 결과를 평가하는 작업을 반복하면서 변수를 선택해 가는 방법
예측 정확도 측면에서 가장 좋은 성능을 보이는 Feature subset을  뽑아내는 방법
Backward elimination
Forward selection
stepwise selection
- 3) Embedded Method :Filer Method와 Wrapper Method을 결합한 방법
각각의 Feature를 직접 학습하며, 모델의 정확도에 기여하는 Feature를 선택
어떤 변수가 가장 크게 기여하는지를 찾아내는 방법으로 overfitting 을 줄이기 위해 모델내부 자체적으로 벌점을 가하는 방식이 사용됨
LASSO Regression (L1 Regression)
Ridge Regression (L2 Regression)
Elastic Net 등

### 2. 본 프로젝트 : Feature_selcetion 개괄

- train data기준으로 진행 : 원저라이징 후 데이터 적용
- stepwise와 lasso로 진행
- 결과 2개의 선택된 피처중 동시에 선택된 피처만 사용

#### 2-1 라이브러리

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 한글폰트 출력
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings 
warnings.filterwarnings('ignore')

# 통계 라이브러리리
from scipy.stats import norm
from scipy import stats
from statsmodels.formula.api import ols
from scipy.stats import kstest

### 2-2 train data 불러오기

In [2]:
df_train = pd.read_csv("./2. train_df.csv")
df_train

,종목코드,회사명,결산월,결산년도,유동비율,당좌비율,부채비율,자기자본비율,차입금의존도,현금비율,...,자기자본이익률,총자산이익률,자산성장률,자기자본성장률,매출총이익성장률,총자산회전율,재고자산회전율,순운전자본회전율,총부채상환능력비율,label
0,[054620],DRB동일,12,2015,104.10,76.63,59.10,40.90,144.51,3.59,...,2.29,0.99,23.66,9.57,47.87,0.00,0.00,0.00,-20.90,0
1,[211270],DRB동일,12,2015,809.29,745.58,10.76,89.24,12.05,424.75,...,6.63,5.61,15.53,30.17,19.01,23.21,310.84,91.50,101.53,1
2,[027410],DRB동일,12,2015,134.06,125.91,51.21,48.79,104.96,3.11,...,0.00,0.00,23.35,29.25,11.81,53.80,1941.99,1592.22,48.50,0
3,[079160],DRB동일,12,2015,67.05,65.71,57.09,42.91,133.05,26.78,...,2.90,1.22,5.73,10.38,5.53,18.95,6299.00,-517.47,18.71,0
4,[011150],DRB동일,12,2015,52.81,18.97,46.57,53.43,87.15,3.43,...,1.31,0.66,-7.50,5.68,-1.05,41.51,221.67,-926.66,71.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7353,[024060],휴림로봇,12,2020,1108.08,1013.02,4.92,95.08,5.18,56.54,...,1.53,1.43,0.07,3.38,-22.44,0.00,0.00,0.00,41.76,0
7354,[010240],휴림로봇,12,2020,212.81,144.51,27.19,72.81,37.34,71.78,...,3.30,2.44,7.72,4.55,18.01,0.00,0.00,0.00,36.28,0
7355,[189980],휴림로봇,12,2020,167.57,113.26,26.38,73.62,35.83,43.82,...,1.66,1.20,4.12,7.90,-15.70,0.00,0.00,0.00,22.10,0
7356,[003280],휴림로봇,12,2020,35.11,29.94,88.54,11.46,3.90,2.28,...,10.73,-2.79,-26.33,-72.95,-322.19,6.36,283.00,-55.78,-0.47,0


### 2-3 Data Scaling : std스케일링 진행

- 데이터 스케일링(Data Scaling)이란 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업을 의미합니다.
- 스케일링의 차이로 인한 변수 영향력 제거하는 과정
- 1) StandardScaler() : 모든 피처들을 평균이 0, 분산이 1인 정규분포를 갖도록 만들어준다.
- 2) MinMaxScaler() : 모든 피처들이 0과 1사이의 데이터값을 갖도록 만들어준다.
- 3) RobustScaler() : StandardScaler와 유사한데 RobustScaler는 중간값(median)과 사분위값(quartile)을 사용한다.
- 4) MaxAbsScaler() : MinMaxScaler와 비슷한데 모든 피처들의 절댓값이 0과 1 사이에 놓이도록 만들어준다.즉, 0을 기준으로 절댓값이 가장 큰 수가 1또는 -1의 값을 가지게 됩니다.

- 데이터 값만 진행
- standard scaler로 변환

In [3]:
# 재무비율 데이터만 부분적으로 추출하기
# 데이터프레임 df_train에서 특정 열만 추출하여 새로운 데이터프레임 df_train_col을 생성하는 작업
# 1.col = df_train.columns[4:19] : df_train.columns: df_train의 열 이름(컬럼 이름)을 가져옵니다.
# [4:19]: 열의 4번 인덱스부터 18번 인덱스까지의 열 이름을 선택합니다.
# Python의 슬라이싱 규칙에 따라 끝 인덱스(19)는 포함되지 않습니다.
# 결과: col에는 선택된 열 이름들의 리스트 또는 Pandas Index 객체가 저장됩니다.
# 2. df_train_col = df_train[col]
# df_train[col]: df_train에서 col에 포함된 열들만 추출합니다.
# 결과: 새로운 데이터프레임 df_train_col은 선택된 열들로만 구성됩니다.

col = df_train.columns[4:19]
df_train_col =  df_train[col]
df_train_col

,유동비율,당좌비율,부채비율,자기자본비율,차입금의존도,현금비율,순운전자본비율,자기자본이익률,총자산이익률,자산성장률,자기자본성장률,매출총이익성장률,총자산회전율,재고자산회전율,순운전자본회전율
0,104.10,76.63,59.10,40.90,144.51,3.59,2.10,2.29,0.99,23.66,9.57,47.87,0.00,0.00,0.00
1,809.29,745.58,10.76,89.24,12.05,424.75,70.17,6.63,5.61,15.53,30.17,19.01,23.21,310.84,91.50
2,134.06,125.91,51.21,48.79,104.96,3.11,13.67,0.00,0.00,23.35,29.25,11.81,53.80,1941.99,1592.22
3,67.05,65.71,57.09,42.91,133.05,26.78,-7.63,2.90,1.22,5.73,10.38,5.53,18.95,6299.00,-517.47
4,52.81,18.97,46.57,53.43,87.15,3.43,-18.93,1.31,0.66,-7.50,5.68,-1.05,41.51,221.67,-926.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7353,1108.08,1013.02,4.92,95.08,5.18,56.54,30.09,1.53,1.43,0.07,3.38,-22.44,0.00,0.00,0.00
7354,212.81,144.51,27.19,72.81,37.34,71.78,26.67,3.30,2.44,7.72,4.55,18.01,0.00,0.00,0.00
7355,167.57,113.26,26.38,73.62,35.83,43.82,16.85,1.66,1.20,4.12,7.90,-15.70,0.00,0.00,0.00
7356,35.11,29.94,88.54,11.46,3.90,2.28,-33.27,10.73,-2.79,-26.33,-72.95,-322.19,6.36,283.00,-55.78


In [4]:
# 스케일링 라이브러리 불러오기
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성: 데이터의 평균을 0, 표준편차를 1로 맞추는 표준화
scaler = StandardScaler()

# 표준화 과정에 사용할 데이터에 대해 fit()을 적용하여 평균과 표준편차 계산
scaler.fit(df_train_col)

# 데이터를 변환하여 표준화된 값으로 변환: 각 값에서 평균을 빼고 표준편차로 나누어 줍니다.
train_std = scaler.transform(df_train_col)

# 표준화된 데이터를 DataFrame으로 변환하여 원래의 컬럼명 유지
train_std = pd.DataFrame(train_std, columns=df_train_col.columns)


In [5]:
# 데이터 확인
train_std

,유동비율,당좌비율,부채비율,자기자본비율,차입금의존도,현금비율,순운전자본비율,자기자본이익률,총자산이익률,자산성장률,자기자본성장률,매출총이익성장률,총자산회전율,재고자산회전율,순운전자본회전율
0,-0.552188,-0.529339,1.093376,-1.093376,0.641550,-0.575705,-0.674772,0.472345,0.370513,0.510124,-0.092690,0.303996,-0.774622,-0.247650,-0.091002
1,1.520270,1.660240,-1.348402,1.348401,-0.765231,3.365105,2.047060,1.189586,2.005592,0.211744,0.351312,0.047599,1.022506,-0.089683,0.053677
2,-0.464140,-0.368037,0.694832,-0.694832,0.221512,-0.580197,-0.212136,0.093894,0.020139,0.498746,0.331482,-0.016366,3.391059,0.739257,2.426601
3,-0.661073,-0.565081,0.991846,-0.991846,0.519840,-0.358715,-1.063833,0.573156,0.451913,-0.147926,-0.075231,-0.072159,0.692658,2.953464,-0.909221
4,-0.702923,-0.718069,0.460454,-0.460454,0.032363,-0.577202,-1.515672,0.310388,0.253722,-0.633480,-0.176533,-0.130616,2.439457,-0.134999,-1.556229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7353,2.398374,2.535613,-1.643395,1.643394,-0.838193,-0.080250,0.444430,0.346746,0.526235,-0.355654,-0.226106,-0.320648,-0.774622,-0.247650,-0.091002
7354,-0.232704,-0.307157,-0.518480,0.518479,-0.496640,0.062351,0.307679,0.639261,0.883687,-0.074891,-0.200888,0.038715,-0.774622,-0.247650,-0.091002
7355,-0.365659,-0.409443,-0.559395,0.559395,-0.512677,-0.199272,-0.084982,0.368230,0.444835,-0.207014,-0.128684,-0.260769,-0.774622,-0.247650,-0.091002
7356,-0.754941,-0.682162,2.580466,-2.580466,-0.851787,-0.587963,-2.089068,1.867163,-0.967279,-1.324561,-1.871281,-2.983668,-0.282173,-0.103832,-0.179201


### 3. 피쳐선택 

### 3-1 LASSO : (Least Absolute Shrinkage and Selection Operator)
- 라쏘는 거기에 추가적인 제약 조건을 준다.(벌점회귀)
- MSE가 최소가 되게 하는 가중치와 편향을 찾으면서 동시에, 가중치들의 절댓값의 합이 최소가 되게 한다는 것이다.
- 가중치의 모든 원소가 0이 되거나 0에 가깝게 되도록 해야 한다.
- 영향력이 적은 피쳐의 경우 제거가 되기도 한다.
- L1-norm이다

In [6]:
# 필요한 라이브러리 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# 로지스틱 회귀 모델 객체 생성
# lr_clf는 로지스틱 회귀 모델의 인스턴스입니다.
# 생성된 객체는 모델의 학습(fit), 예측(predict), 평가(score) 등에 사용됩니다.
# 주요 메서드:
# lr_clf.fit(X, y): 모델 학습.
# lr_clf.predict(X): 클래스 예측.
# lr_clf.predict_proba(X): 각 클래스에 속할 확률 예측.
# lr_clf.score(X, y): 모델 평가(정확도 계산).
    
lr_clf = LogisticRegression()  # Scikit-learn 라이브러리에서 제공하는 로지스틱 회귀 모델 클래스

# 피처(feature)와 타겟(target) 데이터 설정
feature = train_std  # 표준화된 특성 데이터
target = df_train['label']  # 라벨(target) 데이터

# Lasso(라쏘) 모델을 사용하여 특성 선택
# LogisticRegression을 L1 규제를 적용하여 피처 중요도를 평가

# C=0.04: 규제 강도(Regularization Strength)를 조절하는 하이퍼파라미터입니다.
# 값이 작을수록 규제가 강해지고, 특정 특성의 계수를 0으로 만들 가능성이 높아집니다(L1 규제의 효과).
# penalty='l1':  L1 정규화를 사용합니다. 이는 계수(weight) 중 불필요한 값을 0으로 만듭니다.
# solver='liblinear': L1 정규화를 지원하는 최적화 알고리즘입니다. 작은 데이터셋에 적합합니다.
# 2. SelectFromModel: 모델 기반으로 특성 선택을 수행하는 Scikit-learn 클래스
# 특정 모델(여기서는 LogisticRegression)의 계수(coefficient)나 중요도(feature importance) 값을 기반으로 특성을 선택합니다.
# 기능:# 모델이 학습된 후, 중요도가 낮거나 0에 가까운 특성을 자동으로 제거합니다.
# threshold 매개변수를 사용해 중요도를 기준으로 특성을 선택할 수도 있습니다. 기본값은 mean입니다.

lasso = SelectFromModel(LogisticRegression(C=0.04, penalty='l1', solver='liblinear'))

# 모델 학습 (특성 선택 기준을 찾기 위해 사용)
lasso.fit(feature, target)

# 선택된 특성 확인 (True 값만 출력)
lasso_support = lasso.get_support()  # 선택된 특성에 대한 부울 배열 반환
lasso_feature = feature.loc[:, lasso_support].columns.tolist()  # 선택된 특성들의 컬럼명 리스트


In [7]:
# 선택된 피처 확인
lasso_feature

['유동비율', '순운전자본비율', '자기자본이익률', '총자산이익률', '자기자본성장률', '매출총이익성장률', '총자산회전율']

### 3-2 후진제거법(Backward Elimination)
- 전체 변수부터 변수 개수를 제거해가며 성능지표를 비교해가는 방법

In [8]:
import pandas as pd
import statsmodels.api as sm

# X (독립변수)와 y (종속변수)로 분리
X = train_std  # 표준화된 데이터에서 독립 변수 (피처) 선택
y = df_train['label']  # 라벨(종속 변수)

# 상수 항 추가 (절편 항)
X = sm.add_constant(X)  # OLS 모델에 절편을 포함시키기 위해 상수 항 추가

# OLS(최소자승법) 모델 학습
model = sm.OLS(y, X).fit()  # OLS 모델을 학습시키고, 피팅된 결과를 저장

# Stepwise 방법: p-value가 0.05보다 큰 변수 제거 (후진 제거법)
while True:
    # 각 변수의 p-value 확인
    p_values = model.pvalues  # 모델의 p-value 반환
    max_p_value = p_values.max()  # 가장 큰 p-value 값 추출

    # p-value가 0.05보다 크면 해당 변수 제거
    if max_p_value > 0.05:
        excluded_variable = p_values.idxmax()  # p-value가 가장 큰 변수의 이름 추출
        X = X.drop(columns=[excluded_variable])  # 해당 변수를 X에서 제거
        model = sm.OLS(y, X).fit()  # 모델 재학습
    else:
        break  # p-value가 모두 0.05 이하일 경우 반복 종료

# 최종 선택된 변수들만 출력
print("최종 선택된 피처들:")
print(X.columns)  # 최종 선택된 변수들의 컬럼명 출력


최종 선택된 피처들:
Index(['const', '유동비율', '당좌비율', '자기자본이익률', '총자산이익률', '자기자본성장률', '총자산회전율'], dtype='object')


### 3-3 샐랙션 결과

- Lasso

['유동비율', '순운전자본비율', '자기자본이익률', '총자산이익률', '자기자본성장률', '매출총이익성장률', '총자산회전율']

- stepwise(전진)

['유동비율', '당좌비율', '자기자본이익률', '총자산이익률', '자기자본성장률', '총자산회전율']

- 최종피처 : 2방법 중 공통 피처 선택

["유동비율", "자기자본이익률", "자기자본성장률", "총자산회전율", "총자산이익률"]

### 4. train test데이터 피처 선정 데이터로 저장

In [12]:
# 데이터 분할된 train, test data 불러오기
df_train = pd.read_csv("./2. train_df.csv")
df_test = pd.read_csv("./2. test_df.csv")

In [13]:
# 선택된 피처만 저장
df_train_data = df_train[["유동비율", "자기자본이익률", "자기자본성장률", "총자산회전율", "매출총이익성장률","총자산이익률","label"]]
df_test_data = df_test[["유동비율", "자기자본이익률", "자기자본성장률", "총자산회전율", "매출총이익성장률","총자산이익률","label"]]

In [14]:
# 데이터 저장
df_train_data.to_csv("./4. train_data.csv",index=None) # 최종선택된 피처 train data 저장 (index=None : 인덱스를 제외하고 데이터만 저장)
df_test_data.to_csv("./4. test_data.csv",index=None) # 최종선택된 피처 test data 저장 (index=None : 인덱스를 제외하고 데이터만 저장)

### 5. 선택된 피처들의 기초통계량

In [15]:
df_train_data[df_train_data["label"]==0].describe()

,유동비율,자기자본이익률,자기자본성장률,총자산회전율,매출총이익성장률,총자산이익률,label
count,6967.000000,6967.000000,6967.000000,6967.000000,6967.000000,6967.000000,6967.0
mean,294.246432,-0.242486,13.640477,10.172686,13.535545,0.086538,0.0
std,340.651796,5.408744,44.459631,13.030840,108.980631,2.608686,0.0
min,28.600000,-34.340000,-72.950000,0.000000,-322.190000,-12.950000,0.0
25%,104.615000,-0.425000,-0.820000,0.000000,-17.450000,-0.265000,0.0
50%,171.530000,0.500000,4.650000,0.000000,1.730000,0.300000,0.0
75%,327.830000,1.970000,13.850000,18.660000,21.565000,1.250000,0.0
max,2029.620000,10.730000,309.280000,53.800000,760.970000,6.400000,0.0


In [16]:
df_train_data[df_train_data["label"]==1].describe()

,유동비율,자기자본이익률,자기자본성장률,총자산회전율,매출총이익성장률,총자산이익률,label
count,391.000000,391.000000,391.000000,391.000000,391.00000,391.000000,391.0
mean,251.815396,-6.370921,17.968261,7.003581,15.73087,-2.612813,1.0
std,331.649544,11.515422,72.723405,10.220644,163.99612,4.707177,0.0
min,28.600000,-34.340000,-72.950000,0.000000,-322.19000,-12.950000,1.0
25%,88.205000,-11.220000,-9.900000,0.000000,-41.64000,-5.425000,1.0
50%,149.240000,-0.950000,4.240000,0.000000,-4.15000,-0.650000,1.0
75%,259.840000,0.970000,25.350000,12.880000,25.54500,0.540000,1.0
max,2029.620000,10.730000,309.280000,53.800000,760.97000,6.400000,1.0
